In [2]:
# Test script for execute_kql_query function
from azure.identity import DefaultAzureCredential
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from utils.kql_query import execute_kql_query
from dotenv import load_dotenv
import os
import pandas as pd

# Initialize credentials and client
load_dotenv()
credential = DefaultAzureCredential()
client = LogsQueryClient(credential=credential)
workspace_id = os.getenv("SENTINEL_WORKSPACE_ID")

# Check if workspace_id is set, if set then good to go, else raise error
if not workspace_id:
    raise ValueError("SENTINEL_WORKSPACE_ID environment variable is not set.")


In [17]:
rule_title = "Process Chain Analysis"
rule_description = "Analyzes process execution chains for a specific user and/or device to identify unusual parent-child process relationships and execution patterns. Useful for detecting process injection, living-off-the-land techniques, and abnormal process spawning behavior."
rule_file_name = "process_chain_analysis.yaml"
rule_save_path = os.path.join("queries", "analysis", "xdr", rule_file_name) 
rule_references = [
    "https://www.mitre.org/sites/default/files/2021-11/prs-19-3892-ttp-based-hunting.pdf",
    "https://www.cyber.gov.au/about-us/view-all-content/alerts-and-advisories/identifying-and-mitigating-living-off-the-land-techniques"
]
rule_author = "Kevin Flint"
rule_tags = [
    "attack.execution",
    "attack.defense-evasion", 
    "attack.privilege-escalation",
    "attack.t1055",  # Process Injection
    "attack.t1059",  # Command and Scripting Interpreter
    "attack.t1106"   # Native API
]
rule_table = "DeviceProcessEvents"
rule_category = "process_creation"
rule_false_positives = [
    "Reduce Using Baseline",
    "Legitimate administrative activity",
    "Software installations and updates",
    "Scheduled tasks and maintenance scripts"
]
rule_level = "low" 
rule_kql_query = """
DeviceProcessEvents
| where TimeGenerated between ({{ start_time }} .. {{ end_time }})
{% if device_name %}| where DeviceName contains "{{ device_name }}"
{% endif %}{% if user_name %}| where AccountName contains "{{ user_name }}"
{% endif %}| extend Combined = strcat_delim(":", AccountName, InitiatingProcessParentFileName, InitiatingProcessFileName, FileName)
| summarize Count=count(), LastExecutionTime=max(Timestamp) by AccountName, InitiatingProcessParentFileName, InitiatingProcessFileName, FileName, Combined 
| sort by Count desc
"""

In [18]:
import uuid
from datetime import datetime
import yaml

# Build the YAML structure as a dictionary first
analytic_rule_dict = {
    'title': rule_title,
    'id': str(uuid.uuid4()),
    'status': 'test',
    'description': rule_description,
    'references': rule_references,
    'author': rule_author,
    'date': datetime.now().strftime('%Y-%m-%d'),
    'modified': datetime.now().strftime('%Y-%m-%d'),
    'tags': rule_tags,
    'logsource': {
        'product': 'windows',
        'table': rule_table,
        'category': rule_category
    },
    'kql': rule_kql_query.strip(),
    'falsepositives': rule_false_positives,
    'level': rule_level
}

# Convert to YAML string
analytic_rule_yaml = yaml.dump(analytic_rule_dict, default_flow_style=False, sort_keys=False)

print("✓ Valid YAML")
print("\nGenerated YAML:")

# Parse back to verify
parsed_yaml = yaml.safe_load(analytic_rule_yaml)
print(f"\nTitle: {parsed_yaml['title']}")
print(f"ID: {parsed_yaml['id']}")

# Save analytic rule to file
with open(rule_save_path, 'w') as file:
    file.write(analytic_rule_yaml)
print(f"✓ Analytic rule saved to {rule_save_path}")

✓ Valid YAML

Generated YAML:

Title: Process Chain Analysis
ID: 69e64f51-d680-4870-9b0a-d32ddf242c87
✓ Analytic rule saved to queries\analysis\xdr\process_chain_analysis.yaml


In [13]:
from utils.config_loader import load_config

# Set investigation file path
investigation_file_path = os.path.join("investigations", "rtbt")
# Set investigation config file path
investigation_config_path = os.path.join(investigation_file_path, "config.yaml")

# Print config and kql file paths
print(f"Investigation config file path: {investigation_config_path}")

# Test to ensure config file and KQL query file exist
assert os.path.exists(investigation_config_path), f"Config file not found at {investigation_config_path}"

# Read in config file
config = load_config(str(investigation_config_path))

# Print devicename from config
print(f"Device Name from config: {config['devicename']}")

# Print username from config
print(f"Username from config: {config['username']}")

# Print start_time from config
print(f"Start Time from config: {config['start_time']}")

# Print end_time from config
print(f"End Time from config: {config['end_time']}")


Investigation config file path: investigations\rtbt\config.yaml
Device Name from config: RHIAVD-EISP-8
Username from config: alekoz
Start Time from config: 2025-11-17T14:00:00Z
End Time from config: 2025-11-17T23:00:00Z


In [ ]:
# Render it

In [ ]:
# Test it

In [3]:
# Define and execute query
kql_query = """
DeviceProcessEvents
| sample 10
"""

resp = client.query_workspace(workspace_id, kql_query, timespan=None)

if resp.status == LogsQueryStatus.PARTIAL:
    table = resp.partial_data[0]
elif resp.status == LogsQueryStatus.SUCCESS:
    table = resp.tables[0]
else:
    raise RuntimeError("Query failed")

df = pd.DataFrame(table.rows, columns=table.columns)
df.head()

,TenantId,AccountDomain,AccountName,AccountObjectId,AccountSid,AccountUpn,ActionType,AdditionalFields,AppGuardContainerId,DeviceId,...,ProcessRemoteSessionDeviceName,ProcessRemoteSessionIP,InitiatingProcessSessionId,IsInitiatingProcessRemoteSession,InitiatingProcessRemoteSessionDeviceName,InitiatingProcessRemoteSessionIP,InitiatingProcessUniqueId,ProcessUniqueId,SourceSystem,Type
0,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,711c7de5324016bb0face48b338c3fe467923c4d,...,,,0,False,,,12103423998558271,12103423998566239,,DeviceProcessEvents
1,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,c3b4e2b38bfc3e3bab76b7eb6c1f26b9a078648c,...,,,0,False,,,10977524091726953,10977524091726955,,DeviceProcessEvents
2,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,711c7de5324016bb0face48b338c3fe467923c4d,...,,,0,False,,,12103423998558219,12103423998566241,,DeviceProcessEvents
3,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,d8bcc72637cbfa16b8512c167e0961cf4d389ea8,...,,,0,False,,,,,,DeviceProcessEvents
4,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,8fd1c882018b1156f864c5a91f0b408ffa8f9784,...,,,0,False,,,,,,DeviceProcessEvents
